In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2 #opencv
import io
import time
import pandas as pd
import numpy as np
from IPython.display import clear_output
from random import randint
import os

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

#keras imports
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD , Adam
from keras.callbacks import TensorBoard
from keras import backend as K
K.tensorflow_backend._get_available_gpus()
from collections import deque
import random
import pickle
from io import BytesIO
import base64
import json

Using TensorFlow backend.


In [2]:
#path variables
#game_url = "https://www.coolmathgames.com/0-worlds-hardest-game/play"
# use if offline
#game_url = "C:/Users/mrchr/OneDrive/Documents/worlds-hardest-game-ai/game/index.html" 

# run first --> cd C:/Users/mrchr/OneDrive/Documents/worlds-hardest-game-ai/ 
# run --> python -m http.server 8080

game_url = "http://localhost:8080/game/" 

chrome_driver_path = "res/chromedriver"
loss_file_path = "./objects/loss_df.csv"
actions_file_path = "./objects/actions_df.csv"
q_value_file_path = "./objects/q_values.csv"

#scripts

#get image from canvas
getbase64Script = "canvasRunner = document.getElementById('twhgCanvas'); \
return canvasRunner.toDataURL().substring(22);"

In [3]:
'''
* Game class: Selenium interfacing between the python and browser
* __init__():  Launch the broswer window using the attributes in chrome_options
* get_crashed() : return true if the agent as crashed on an obstacles. Gets javascript variable from game decribing the state
* get_playing(): true if game in progress, false is crashed or paused
* press_up(): sends a single to press up get to the browser
* pause(): pause the game
* resume(): resume a paused game if not crashed
* end(): close the browser and end the game
'''
class Game:
    def __init__(self,custom_config=True):
        chrome_options = Options()
        chrome_options.add_argument("disable-infobars")
        chrome_options.add_argument("--mute-audio")
        self._driver = webdriver.Chrome(executable_path = chrome_driver_path,chrome_options=chrome_options)
        self._driver.set_window_position(x=-10,y=0)
        self._driver.set_window_size(810, 600)
        self._driver.get(game_url)
    def get_crashed(self):
        return self._driver.execute_script("return player.dying")
    def get_playing(self):
        return self._driver.execute_script("return state == 'game' && !player.dying")
    def get_completed(self):
        if self._driver.execute_script("return finishLevelTimer > 0"):
            self._driver.execute_script("finishLevelTimer = 1")
            return True
        return False
    def press_up(self, stride):
        start_y = self._driver.execute_script("return player.y")
        prev_y = -1
        curr_y = start_y
        while curr_y >= start_y - stride:
            if curr_y == prev_y:
                self._driver.execute_script("keydown.up = false")
                self._driver.execute_script("keyUp = false")
                return True
            prev_y = curr_y
            self._driver.execute_script("keyUp = true")
            self._driver.execute_script("keydown.up = true")
            time.sleep(0.025)
            curr_y = self._driver.execute_script("return player.y")
        self._driver.execute_script("keydown.up = false")
        self._driver.execute_script("keyUp = false")
        return False
    def press_down(self, stride):
        start_y = self._driver.execute_script("return player.y")
        prev_y = -1
        curr_y = start_y
        while curr_y < start_y + stride:
            if curr_y == prev_y:
                self._driver.execute_script("keydown.down = false")
                self._driver.execute_script("keyDown = false")
                return True
            prev_y = curr_y
            self._driver.execute_script("keyDown = true")
            self._driver.execute_script("keydown.down = true")
            time.sleep(0.025)
            curr_y = self._driver.execute_script("return player.y")
        self._driver.execute_script("keydown.down = false")
        self._driver.execute_script("keyDown = false")
        return False
    def press_left(self, stride):
        start_x = self._driver.execute_script("return player.x")
        prev_x = -1
        curr_x = start_x
        while curr_x >= start_x - stride:
            if curr_x == prev_x:
                self._driver.execute_script("keydown.left = false")
                self._driver.execute_script("keyLeft = false")
                return True
            prev_x = curr_x
            self._driver.execute_script("keyLeft = true")
            self._driver.execute_script("keydown.left = true")
            time.sleep(0.025)
            curr_x = self._driver.execute_script("return player.x")
        self._driver.execute_script("keydown.left = false")
        self._driver.execute_script("keyLeft = false")
        return False
    def press_right(self, stride): # 120ms to 50ms to move
        start_x = self._driver.execute_script("return player.x")
        prev_x = -1
        curr_x = start_x
        while curr_x < start_x + stride:
            if curr_x == prev_x:
                self._driver.execute_script("keydown.right = false")
                self._driver.execute_script("keyRight = false")
                return True
            prev_x = curr_x
            self._driver.execute_script("keyRight = true")
            self._driver.execute_script("keydown.right = true")
            time.sleep(0.025)
            curr_x = self._driver.execute_script("return player.x")
        self._driver.execute_script("keydown.right = false")
        self._driver.execute_script("keyRight = false")
        return False
    def pause(self):
        return self._driver.execute_script("paused = true")
    def resume(self):
        return self._driver.execute_script("paused = false")
    def end(self):
        self._driver.close()
    def enter_level(self):
        time.sleep(0.50) # wait for things to load
        while self._driver.execute_script("return state !== 'game'"):
            self._driver.find_element_by_id("twhgCanvas").click()

In [29]:
game = Game()
agent = Agent(game)
game_state = Game_sate(agent,game)
game.enter_level()

C:\Users\mrchr\Miniconda3\envs\tensorflow_gpuenv\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: use options instead of chrome_options
  app.launch_new_instance()


In [10]:
agent.up()
agent.down()
agent.left()
agent.right()

False

In [91]:
agent.down()

False

In [465]:
agent.up()

False

In [4]:
class Agent:
    def __init__(self,game): #takes game as input for taking actions
        self._game = game;
        self.stride = 32
    def is_running(self):
        return self._game.get_playing()
    def is_crashed(self):
        return self._game.get_crashed()
    def won(self):
        return self._game.get_completed()
    def up(self):
        return self._game.press_up(self.stride)
    def down(self):
        return self._game.press_down(self.stride)
    def left(self):
        return self._game.press_left(self.stride)
    def right(self):
        return self._game.press_right(self.stride)

In [33]:
class Game_sate:
    def __init__(self,agent,game):
        self._agent = agent
        self._game = game
        self._display = show_img() #display the processed image on screen using openCV, implemented using python coroutine 
        self._display.__next__() # initiliaze the display coroutine
    def get_block_map(self):
        screen = grab_screen(self._game._driver)
        hsv = cv2.cvtColor(screen, cv2.COLOR_BGR2HSV)
        block_map = np.zeros([13, 20])

        # purplish outside
        mask1 = cv2.inRange(hsv, (95, 50, 50), (135, 255, 255))
        grey_img = cv2.cvtColor(cv2.bitwise_and(screen, screen, mask=mask1), cv2.COLOR_RGB2GRAY)

        for r in range(grey_img.shape[0]//4):
            for c in range(grey_img.shape[1]//4):
                if np.any(grey_img[r*4:r*4+4, c*4:c*4+4] >= 128):
                    block_map[r, c] = -1

        # red
        mask2 = cv2.inRange(hsv, (0, 50, 50), (18, 255, 255))
        mask2 = cv2.bitwise_or(mask2, cv2.inRange(hsv, (165, 50, 50), (180, 255, 255)))
        grey_img = cv2.cvtColor(cv2.bitwise_and(screen, screen, mask=mask2), cv2.COLOR_RGB2GRAY)

        for r in range(grey_img.shape[0]//4):
            for c in range(grey_img.shape[1]//4):
                if np.any(grey_img[r*4:r*4+4, c*4:c*4+4] >= 25) and block_map[r, c] == 0:
                    block_map[r, c] = 1

        goal_location = self._game._driver.execute_script("return checkpoints[level][checkpoints[level].length - 1]")

        for c in range(goal_location[0], goal_location[0] + goal_location[2]):
            for r in range(goal_location[1] - 1, goal_location[1] - 1 + goal_location[3]):
                block_map[r, c] = 100

        return block_map
    def get_state(self,actions):
        actions_df.loc[len(actions_df)] = actions[1] # storing actions in a dataframe
        score = 1
        action_reward = -0.2
        is_over = False #game over
        if actions[1] == 1:
            if self._agent.up():
                action_reward = -1
            else:
                action_reward = -0.1
        elif actions[2] == 1:
            if self._agent.down():
                action_reward = -1
            else:
                action_reward = -0.1
        elif actions[3] == 1:
            if self._agent.left():
                action_reward = -1
            else:
                action_reward = -0.1
        elif actions[4] == 1:
            if self._agent.right():
                action_reward = -1
            else:
                action_reward = -0.1
        
        image = self.get_block_map()
        self._display.send(image) #display the image on screen
        
        if self._agent.is_crashed():
            action_reward = -100
            is_over = True
        elif self._agent.won():
            action_reward = 100
            print("WE JUST WON")
            is_over = True
        
        return image, action_reward , is_over #return the Experience tuple

In [6]:
def save_obj(obj, name ):
    with open('objects/'+ name + '.pkl', 'wb') as f: #dump files into objects folder
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
def load_obj(name ):
    with open('objects/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

def grab_screen(_driver):
    image_b64 = _driver.execute_script(getbase64Script)
    screen = np.array(Image.open(BytesIO(base64.b64decode(image_b64))))
    image = process_img(screen)#processing image as required
    return image

def process_img(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #BGR to RGB
    image = image[50:550, :] #Crop Region of Interest(ROI)
    image = cv2.resize(image, (80, 52))
    return  image

def show_img(graphs = False):
    """
    Show images in new window
    """
    while True:
        screen = (yield)
        window_title = "logs" if graphs else "game_play"
        cv2.namedWindow(window_title, cv2.WINDOW_NORMAL)
        cv2.resizeWindow(window_title, 200, 130)
        cv2.imshow(window_title, screen)
        if (cv2.waitKey(1) & 0xFF == ord('q')):
            cv2.destroyAllWindows()
            break

In [89]:
cv2.destroyAllWindows()

In [7]:
#Intialize log structures from file if exists else create new
loss_df = pd.read_csv(loss_file_path) if os.path.isfile(loss_file_path) else pd.DataFrame(columns =['loss'])
actions_df = pd.read_csv(actions_file_path) if os.path.isfile(actions_file_path) else pd.DataFrame(columns = ['actions'])
q_values_df =pd.read_csv(actions_file_path) if os.path.isfile(q_value_file_path) else pd.DataFrame(columns = ['qvalues'])

In [8]:
#game parameters
ACTIONS = 5 # possible actions: up, down, left, right, do nothing
GAMMA = 0.99 # decay rate of past observations original 0.99
OBSERVATION = 100. # timesteps to observe before training
EXPLORE = 100000  # frames over which to anneal epsilon
FINAL_EPSILON = 0.0001 # final value of epsilon
INITIAL_EPSILON = 0.1 # starting value of epsilon
REPLAY_MEMORY = 50000 # number of previous transitions to remember
BATCH = 16 # size of minibatch
FRAME_PER_ACTION = 1
LEARNING_RATE = 1e-4
img_rows , img_cols = 20, 13
img_channels = 4 #We stack 4 frames

In [21]:
# training variables saved as checkpoints to filesystem to resume training from the same step
def init_cache():
    """initial variable caching, done only once"""
    save_obj(INITIAL_EPSILON,"epsilon")
    t = 0
    save_obj(t,"time")
    D = deque()
    save_obj(D,"D")

In [32]:
'''Call only once to init file structure
'''
init_cache()

In [9]:
def buildmodel():
    print("Now we build the model")
    model = Sequential()
    model.add(Conv2D(32, (8, 8), padding='same',strides=(4, 4),input_shape=(img_cols,img_rows,img_channels)))  #100*63*4
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Activation('relu'))
    #model.add(Conv2D(64, (4, 4),strides=(2, 2),  padding='same'))
    #model.add(MaxPooling2D(pool_size=(2,2)))
    #model.add(Activation('relu'))
    #model.add(Conv2D(64, (3, 3),strides=(1, 1),  padding='same'))
    #model.add(MaxPooling2D(pool_size=(2,2)))
    #model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(ACTIONS))
    adam = Adam(lr=LEARNING_RATE)
    model.compile(loss='mse',optimizer=adam)
    
    #create model file if not present
    if not os.path.isfile(loss_file_path):
        model.save_weights('model.h5')
    print("We finish building the model")
    return model

In [10]:
''' 
main training module
Parameters:
* model => Keras Model to be trained
* game_state => Game State module with access to game environment and dino
* observe => flag to indicate wherther the model is to be trained(weight updates), else just play
'''
def trainNetwork(model,game_state,observe=False):
    last_time = time.time()
    # store the previous observations in replay memory
    D = load_obj("D") #load from file system
    # get the first state by doing nothing
    do_nothing = np.zeros(ACTIONS)
    do_nothing[0] =1 #0 => do nothing,
                     #1=> jump
    
    x_t, r_0, terminal = game_state.get_state(do_nothing) # get next step after performing the action
    

    s_t = np.stack((x_t, x_t, x_t, x_t), axis=2) # stack 4 images to create placeholder input
    

    
    s_t = s_t.reshape(1, s_t.shape[0], s_t.shape[1], s_t.shape[2])  #1*20*40*4
    
    initial_state = s_t 

    if observe :
        OBSERVE = 999999999    #We keep observe, never train
        epsilon = FINAL_EPSILON
        print ("Now we load weight")
        model.load_weights("model.h5")
        adam = Adam(lr=LEARNING_RATE)
        model.compile(loss='mse',optimizer=adam)
        print ("Weight load successfully")    
    else:                       #We go to training mode
        OBSERVE = OBSERVATION
        epsilon = load_obj("epsilon") 
        model.load_weights("model.h5")
        adam = Adam(lr=LEARNING_RATE)
        model.compile(loss='mse',optimizer=adam)

    t = load_obj("time") # resume from the previous time step stored in file system
    while (True): #endless running
        
        loss = 0
        Q_sa = 0
        action_index = 0
        r_t = 0 #reward at 4
        a_t = np.zeros([ACTIONS]) # action at t
        
        #choose an action epsilon greedy
        if t % FRAME_PER_ACTION == 0: #parameter to skip frames for actions
            if  random.random() <= epsilon: #randomly explore an action
                print("----------Random Action----------")
                action_index = random.randrange(ACTIONS)
                a_t[action_index] = 1
            else: # predict the output
                q = model.predict(s_t)       #input a stack of 4 images, get the prediction
                max_Q = np.argmax(q)         # chosing index with maximum q value
                action_index = max_Q 
                a_t[action_index] = 1        # o=> do nothing, 1=> jump
                
        #We reduced the epsilon (exploration parameter) gradually
        if epsilon > FINAL_EPSILON and t > OBSERVE:
            epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / EXPLORE 

        #run the selected action and observed next state and reward
        x_t1, r_t, terminal = game_state.get_state(a_t)
        print('fps: {0}'.format(1 / (time.time()-last_time))) # helpful for measuring frame rate
        last_time = time.time()
        x_t1 = x_t1.reshape(1, x_t1.shape[0], x_t1.shape[1], 1) #1x20x40x1
        s_t1 = np.append(x_t1, s_t[:, :, :, :3], axis=3) # append the new image to input stack and remove the first one
        
        
        # store the transition in D
        D.append((s_t, action_index, r_t, s_t1, terminal))
        if len(D) > REPLAY_MEMORY:
            D.popleft()

        #only train if done observing
        if t > OBSERVE: 
            
            #sample a minibatch to train on
            minibatch = random.sample(D, BATCH)
            inputs = np.zeros((BATCH, s_t.shape[1], s_t.shape[2], s_t.shape[3]))   #32, 20, 40, 4
            targets = np.zeros((inputs.shape[0], ACTIONS))                         #32, 2

            #Now we do the experience replay
            for i in range(0, len(minibatch)):
                state_t = minibatch[i][0]    # 4D stack of images
                action_t = minibatch[i][1]   #This is action index
                reward_t = minibatch[i][2]   #reward at state_t due to action_t
                state_t1 = minibatch[i][3]   #next state
                terminal = minibatch[i][4]   #wheather the agent died or survided due the action
                

                inputs[i:i + 1] = state_t    

                targets[i] = model.predict(state_t)  # predicted q values
                Q_sa = model.predict(state_t1)      #predict q values for next step
                
                if terminal:
                    targets[i, action_t] = reward_t # if terminated, only equals reward
                else:
                    targets[i, action_t] = reward_t + GAMMA * np.max(Q_sa)

            loss += model.train_on_batch(inputs, targets)
            loss_df.loc[len(loss_df)] = loss
            q_values_df.loc[len(q_values_df)] = np.max(Q_sa)
        s_t = initial_state if terminal else s_t1 #reset game to initial frame if terminate
        t = t + 1
        
        # save progress every 1000 iterations
        if t % 1000 == 0:
            print("Now we save model")
            game_state._game.pause() #pause game while saving to filesystem
            model.save_weights("model.h5", overwrite=True)
            save_obj(D,"D") #saving episodes
            save_obj(t,"time") #caching time steps
            save_obj(epsilon,"epsilon") #cache epsilon to avoid repeated randomness in actions
            loss_df.to_csv("./objects/loss_df.csv",index=False)
            actions_df.to_csv("./objects/actions_df.csv",index=False)
            q_values_df.to_csv(q_value_file_path,index=False)
            with open("model.json", "w") as outfile:
                json.dump(model.to_json(), outfile)
            clear_output()
            game_state._game.resume()
        # print info
        state = ""
        if t <= OBSERVE:
            state = "observe"
        elif t > OBSERVE and t <= OBSERVE + EXPLORE:
            state = "explore"
        else:
            state = "train"

        print("TIMESTEP", t, "/ STATE", state,             "/ EPSILON", epsilon, "/ ACTION", action_index, "/ REWARD", r_t,             "/ Q_MAX " , np.max(Q_sa), "/ Loss ", loss)

    print("Episode finished!")
    print("************************")


In [11]:
#main function
def playGame(observe=False):
    game = Game()
    agent = Agent(game)
    game_state = Game_sate(agent, game)
    game.enter_level()
    model = buildmodel()
    try:
        trainNetwork(model,game_state,observe=observe)
    except StopIteration:
        game.end()

In [34]:
playGame(observe=False);

TIMESTEP 28000 / STATE explore / EPSILON 0.07212889899995742 / ACTION 0 / REWARD -0.2 / Q_MAX  -20.012976 / Loss  0.18924832344055176
fps: 0.27484845768374455
TIMESTEP 28001 / STATE explore / EPSILON 0.07212789999995742 / ACTION 0 / REWARD -0.2 / Q_MAX  -20.046127 / Loss  0.050572384148836136
fps: 3.841676512882514
TIMESTEP 28002 / STATE explore / EPSILON 0.07212690099995742 / ACTION 0 / REWARD -0.2 / Q_MAX  -20.06317 / Loss  0.015610002912580967
fps: 5.050883298611529
TIMESTEP 28003 / STATE explore / EPSILON 0.07212590199995741 / ACTION 0 / REWARD -0.2 / Q_MAX  -20.079504 / Loss  0.050195351243019104
fps: 5.490717249865819
TIMESTEP 28004 / STATE explore / EPSILON 0.07212490299995741 / ACTION 0 / REWARD -0.2 / Q_MAX  -20.0782 / Loss  0.01841265708208084
fps: 5.683331052396948
TIMESTEP 28005 / STATE explore / EPSILON 0.07212390399995741 / ACTION 0 / REWARD -0.2 / Q_MAX  -20.10409 / Loss  0.0043601393699646
fps: 5.840835538225874
TIMESTEP 28006 / STATE explore / EPSILON 0.072122904999957

TIMESTEP 28052 / STATE explore / EPSILON 0.07207695099995734 / ACTION 0 / REWARD -0.2 / Q_MAX  24.366228 / Loss  0.0117575004696846
fps: 3.998017340643754
TIMESTEP 28053 / STATE explore / EPSILON 0.07207595199995734 / ACTION 0 / REWARD -0.2 / Q_MAX  24.012728 / Loss  0.018365751951932907
fps: 4.319291333312051
TIMESTEP 28054 / STATE explore / EPSILON 0.07207495299995734 / ACTION 0 / REWARD -0.2 / Q_MAX  -13.9468565 / Loss  0.009118248708546162
fps: 4.9267721409701775
TIMESTEP 28055 / STATE explore / EPSILON 0.07207395399995734 / ACTION 0 / REWARD -0.2 / Q_MAX  -14.043505 / Loss  0.01009825523942709
fps: 4.933894835901659
TIMESTEP 28056 / STATE explore / EPSILON 0.07207295499995733 / ACTION 0 / REWARD -0.2 / Q_MAX  23.716202 / Loss  0.0113978311419487
fps: 6.151313695727097
TIMESTEP 28057 / STATE explore / EPSILON 0.07207195599995733 / ACTION 0 / REWARD -0.2 / Q_MAX  23.388638 / Loss  0.02670690044760704
fps: 6.234018867167354
TIMESTEP 28058 / STATE explore / EPSILON 0.07207095699995733

fps: 6.544274408967222
TIMESTEP 28105 / STATE explore / EPSILON 0.07202400399995726 / ACTION 0 / REWARD -0.2 / Q_MAX  -18.684782 / Loss  0.007780664600431919
fps: 6.431570989798249
TIMESTEP 28106 / STATE explore / EPSILON 0.07202300499995726 / ACTION 0 / REWARD -0.2 / Q_MAX  -18.76381 / Loss  0.010284430347383022
fps: 7.372604165201862
TIMESTEP 28107 / STATE explore / EPSILON 0.07202200599995726 / ACTION 0 / REWARD -0.2 / Q_MAX  -19.715277 / Loss  0.011205269955098629
----------Random Action----------
fps: 5.22180349564072
TIMESTEP 28108 / STATE explore / EPSILON 0.07202100699995725 / ACTION 4 / REWARD -1 / Q_MAX  14.922478 / Loss  0.008248036727309227
fps: 7.772296015180266
TIMESTEP 28109 / STATE explore / EPSILON 0.07202000799995725 / ACTION 0 / REWARD -0.2 / Q_MAX  14.729747 / Loss  0.004142895806580782
fps: 7.26574520459195
TIMESTEP 28110 / STATE explore / EPSILON 0.07201900899995725 / ACTION 0 / REWARD -0.2 / Q_MAX  14.583232 / Loss  0.008617683313786983
fps: 4.685394224893793
TIM

TIMESTEP 28156 / STATE explore / EPSILON 0.07197305499995718 / ACTION 0 / REWARD -0.2 / Q_MAX  10.112855 / Loss  0.003501851810142398
fps: 9.025715183363245
TIMESTEP 28157 / STATE explore / EPSILON 0.07197205599995718 / ACTION 0 / REWARD -0.2 / Q_MAX  10.043454 / Loss  0.0026914861518889666
fps: 9.175579065270085
TIMESTEP 28158 / STATE explore / EPSILON 0.07197105699995718 / ACTION 0 / REWARD -0.2 / Q_MAX  -20.339779 / Loss  0.01009862869977951
fps: 8.546426308207357
TIMESTEP 28159 / STATE explore / EPSILON 0.07197005799995718 / ACTION 0 / REWARD -0.2 / Q_MAX  9.856117 / Loss  0.027552347630262375
fps: 8.595008534958493
TIMESTEP 28160 / STATE explore / EPSILON 0.07196905899995718 / ACTION 0 / REWARD -0.2 / Q_MAX  -20.411255 / Loss  0.0005549761699512601
fps: 6.151313695727097
TIMESTEP 28161 / STATE explore / EPSILON 0.07196805999995717 / ACTION 0 / REWARD -0.2 / Q_MAX  -19.871487 / Loss  0.009728130884468555
fps: 6.499743531333342
TIMESTEP 28162 / STATE explore / EPSILON 0.071967060999

fps: 5.615067746759253
TIMESTEP 28209 / STATE explore / EPSILON 0.0719201079999571 / ACTION 0 / REWARD -0.2 / Q_MAX  -19.870539 / Loss  0.0021577223669737577
fps: 7.6540070220515775
TIMESTEP 28210 / STATE explore / EPSILON 0.0719191089999571 / ACTION 0 / REWARD -0.2 / Q_MAX  -20.29999 / Loss  0.0038840847555547953
fps: 7.323135695167325
TIMESTEP 28211 / STATE explore / EPSILON 0.0719181099999571 / ACTION 0 / REWARD -0.2 / Q_MAX  -0.7719152 / Loss  0.021630723029375076
fps: 7.612609080745162
TIMESTEP 28212 / STATE explore / EPSILON 0.0719171109999571 / ACTION 0 / REWARD -0.2 / Q_MAX  -20.20197 / Loss  0.0056791459210217
fps: 7.682452345322634
TIMESTEP 28213 / STATE explore / EPSILON 0.0719161119999571 / ACTION 0 / REWARD -0.2 / Q_MAX  -19.898952 / Loss  0.006368829868733883
fps: 7.559865323749306
TIMESTEP 28214 / STATE explore / EPSILON 0.07191511299995709 / ACTION 0 / REWARD -0.2 / Q_MAX  -0.84890264 / Loss  0.009480666369199753
fps: 7.478824016090609
TIMESTEP 28215 / STATE explore / E

fps: 6.989842631616663
TIMESTEP 28261 / STATE explore / EPSILON 0.07186815999995702 / ACTION 0 / REWARD -0.2 / Q_MAX  -17.843033 / Loss  3.5151267051696777
fps: 6.580838501861612
TIMESTEP 28262 / STATE explore / EPSILON 0.07186716099995702 / ACTION 0 / REWARD -0.2 / Q_MAX  5.9156284 / Loss  0.03010724112391472
fps: 6.191457765627352
TIMESTEP 28263 / STATE explore / EPSILON 0.07186616199995702 / ACTION 0 / REWARD -0.2 / Q_MAX  -21.213743 / Loss  0.004550321027636528
fps: 6.468518764130545
TIMESTEP 28264 / STATE explore / EPSILON 0.07186516299995702 / ACTION 0 / REWARD -0.2 / Q_MAX  5.1406093 / Loss  0.0402977392077446
fps: 6.4063930985816535
TIMESTEP 28265 / STATE explore / EPSILON 0.07186416399995701 / ACTION 0 / REWARD -0.2 / Q_MAX  4.9010835 / Loss  0.004968602675944567
fps: 4.67016068241312
TIMESTEP 28266 / STATE explore / EPSILON 0.07186316499995701 / ACTION 0 / REWARD -0.2 / Q_MAX  -18.330647 / Loss  0.012765862978994846
fps: 2.764495743486044
TIMESTEP 28267 / STATE explore / EPSI

WebDriverException: Message: chrome not reachable
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.45.615291 (ec3682e3c9061c10f26ea9e5cdcf3c53f3f74387),platform=Windows NT 10.0.17763 x86_64)
